Download individual csv files with ozone levels, temperature, barometric pressure, relative humidity, and wind speed from 2016-2018. Examine data, clean, fill in missing data, export condensed file for future work.

In [43]:
#import necessary packages
import pandas as pd
import numpy as np
from datetime import date

In [44]:
#all csv files were downloaded from: https://aqs.epa.gov/aqsweb/airdata/download_files.html#Raw

ozone2016 = pd.read_csv('Ozone/hourly_44201_2016.csv', parse_dates=[['Date Local', 'Time Local']])
ozone2017 = pd.read_csv('Ozone/hourly_44201_2017.csv', parse_dates=[['Date Local', 'Time Local']])
ozone2018 = pd.read_csv('Ozone/hourly_44201_2018.csv', parse_dates=[['Date Local', 'Time Local']])

In [45]:
temp2016 = pd.read_csv('Ozone/hourly_TEMP_2016.csv', parse_dates=[['Date Local', 'Time Local']])
temp2017 = pd.read_csv('Ozone/hourly_TEMP_2017.csv', parse_dates=[['Date Local', 'Time Local']])
temp2018 = pd.read_csv('Ozone/hourly_TEMP_2018.csv', parse_dates=[['Date Local', 'Time Local']])

In [46]:
press2016 = pd.read_csv('Ozone/hourly_PRESS_2016.csv', parse_dates=[['Date Local', 'Time Local']])
press2017 = pd.read_csv('Ozone/hourly_PRESS_2017.csv', parse_dates=[['Date Local', 'Time Local']])
press2018 = pd.read_csv('Ozone/hourly_PRESS_2018.csv', parse_dates=[['Date Local', 'Time Local']])

In [47]:
relhum2016 = pd.read_csv('Ozone/hourly_RH_DP_2016.csv', parse_dates=[['Date Local', 'Time Local']])
relhum2017 = pd.read_csv('Ozone/hourly_RH_DP_2017.csv', parse_dates=[['Date Local', 'Time Local']])
relhum2018 = pd.read_csv('Ozone/hourly_RH_DP_2018.csv', parse_dates=[['Date Local', 'Time Local']])

In [48]:
wind2016 = pd.read_csv('Ozone/hourly_WIND_2016.csv', parse_dates=[['Date Local', 'Time Local']])
wind2017 = pd.read_csv('Ozone/hourly_WIND_2017.csv', parse_dates=[['Date Local', 'Time Local']])
wind2018 = pd.read_csv('Ozone/hourly_WIND_2018.csv', parse_dates=[['Date Local', 'Time Local']])

In [51]:
wind2018.head(2)

,Date Local_Time Local,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,...,Units of Measure,MDL,Uncertainty,Qualifier,Method Type,Method Code,Method Name,State Name,County Name,Date of Last Change
0,2018-01-01 00:00:00,1,53,1000,61103,1,31.0921,-87.5435,NAD83,Wind Speed - Resultant,...,Knots,0.1,NaN,NaN,Non-FRM,67,Instrumental - RM Young Model 05103,Alabama,Escambia,2018-07-30
1,2018-01-01 01:00:00,1,53,1000,61103,1,31.0921,-87.5435,NAD83,Wind Speed - Resultant,...,Knots,0.1,NaN,NaN,Non-FRM,67,Instrumental - RM Young Model 05103,Alabama,Escambia,2018-07-30


In [55]:
#filter out LA data from site number 1103 only

def get_site(df):
    return df[(df['County Name'] == 'Los Angeles') & (df['Site Num'] == 1103)]
la_wind2016 = get_site(wind2016)
la_wind2017 = get_site(wind2017)
la_wind2018 = get_site(wind2018)
la_wind2018.shape

(17458, 23)

In [56]:
la_press2016= get_site(press2016)
la_press2017= get_site(press2017)
la_press2018= get_site(press2018)
la_press2018.shape

(8731, 23)

In [57]:
la_temp2016= get_site(temp2016)
la_temp2017= get_site(temp2017)
la_temp2018= get_site(temp2018)
la_temp2016.shape

(8776, 23)

In [58]:
la_ozone2016= get_site(ozone2016)
la_ozone2017= get_site(ozone2017)
la_ozone2018= get_site(ozone2018)
la_ozone2016.shape

(8268, 23)

In [59]:
la_relhum2016= get_site(relhum2016)
la_relhum2017= get_site(relhum2017)
la_relhum2018= get_site(relhum2018)
la_relhum2016.shape

(8776, 23)

In [67]:
#combine data from all three years
ozone = pd.concat([la_ozone2016, la_ozone2017, la_ozone2018])
temp = pd.concat([la_temp2016, la_temp2017, la_temp2018])
wind = pd.concat([la_wind2016, la_wind2017, la_wind2018])
press = pd.concat([la_press2016, la_press2017, la_press2018])
relhum = pd.concat([la_relhum2016, la_relhum2017, la_relhum2018])

In [68]:
#some sites have dew point listed in the relative humidity file too
#this site clearly doesn't have dew point listed, but just to be safe and keep consistent
#code for analyzing future cities/monitoring sites
print(relhum['Units of Measure'].unique())
rh = relhum[relhum['Parameter Name'] == 'Relative Humidity ']
dp = relhum[relhum['Parameter Name'] == 'Dew Point']
print(rh.shape)
print(dp.shape)
#expected shape (+24 because of leap year having 24 extra hours)
print(365*24*3)

['Percent relative humidity']
(26260, 23)
(0, 23)
26280


In [69]:
#wind speed and direction both listed - separate out
print(wind['Parameter Name'].unique())
speed = wind[wind['Parameter Name'] == 'Wind Speed - Resultant']
winddir = wind[wind['Parameter Name'] == 'Wind Direction - Resultant']
print(speed['Sample Measurement'].head())
print(winddir['Sample Measurement'].head())

['Wind Speed - Resultant' 'Wind Direction - Resultant']
1517420    3.7
1517421    2.9
1517422    2.6
1517423    3.3
1517424    2.0
Name: Sample Measurement, dtype: float64
1524012    67.0
1524013    60.0
1524014    40.0
1524015    27.0
1524016    28.0
Name: Sample Measurement, dtype: float64


In [70]:
press['Parameter Name'].unique()

array(['Barometric pressure'], dtype=object)

In [71]:
#get hourly data from the first date in 2016 to the last date in 2018
#this is the ideal dataset - use this to fill in data missing from other dataframes
idx_hourly = pd.date_range(start = '2016-01-01 00:00:00', end = '2018-12-31 23:00:00', freq = 'H')

In [ ]:
#rename column to 'Date', set 'Date' as index, then reindex, then forward fill 
#forward fill - use last data point to fill in missing data
def fill_in(df):
    return df.rename(columns = {'Date Local_Time Local':'Date'}).set_index('Date').reindex(idx_hourly, method='ffill')

temp = fill_in(temp)
ozone = fill_in(ozone)
speed = fill_in(speed)
winddir = fill_in(winddir)
press = fill_in(press)
rh = fill_in(rh)

print(temp.shape)
print(ozone.shape)
print(speed.shape)
print(winddir.shape)
print(press.shape)
print(rh.shape)

In [74]:
#extract out sample measurement only now, then merge the dataframes and label the columns
#do this for temp and ozone
temp_combo = temp[['Sample Measurement']]
ozone_combo = ozone[['Sample Measurement']]
merge1 = pd.merge(temp_combo, ozone_combo, left_index = True, right_index = True, how = 'outer')
merge1.columns = ['Temp', 'Ozone']
merge1.head()

,Temp,Ozone
2016-01-01 00:00:00,46.6,0.004
2016-01-01 01:00:00,45.7,0.003
2016-01-01 02:00:00,45.4,0.003
2016-01-01 03:00:00,44.7,0.003
2016-01-01 04:00:00,43.8,0.003


In [75]:
#repeat for wind speed and direction
speed_combo = speed[['Sample Measurement']]
winddir_combo = winddir[['Sample Measurement']]
merge2 = pd.merge(speed_combo, winddir_combo, left_index = True, right_index = True, how = 'outer')
merge2.columns = ['Wind Speed', 'Wind Dir']
merge2.head()

,Wind Speed,Wind Dir
2016-01-01 00:00:00,NaN,NaN
2016-01-01 01:00:00,NaN,NaN
2016-01-01 02:00:00,NaN,NaN
2016-01-01 03:00:00,NaN,NaN
2016-01-01 04:00:00,NaN,NaN


In [76]:
#repeat for pressure and relative humidity
press_combo = press[['Sample Measurement']]
rh_combo = rh[['Sample Measurement']]
merge3 = pd.merge(press_combo, rh_combo, left_index = True, right_index = True, how = 'outer')
merge3.columns = ['Pressure', 'Humidity']
merge3.head()

,Pressure,Humidity
2016-01-01 00:00:00,1007.0,35.0
2016-01-01 01:00:00,1007.0,36.0
2016-01-01 02:00:00,1007.0,33.0
2016-01-01 03:00:00,1007.0,33.0
2016-01-01 04:00:00,1007.0,34.0


In [77]:
#now combine the combined
merge4 = pd.merge(merge1, merge3, left_index = True, right_index = True, how = 'outer')
merge4.head()

,Temp,Ozone,Pressure,Humidity
2016-01-01 00:00:00,46.6,0.004,1007.0,35.0
2016-01-01 01:00:00,45.7,0.003,1007.0,36.0
2016-01-01 02:00:00,45.4,0.003,1007.0,33.0
2016-01-01 03:00:00,44.7,0.003,1007.0,33.0
2016-01-01 04:00:00,43.8,0.003,1007.0,34.0


In [78]:
#combine the combined - wind data is missing for the beginning of 2016. 
#disappointing, but will just need to remove those columns eventually
merge5 = pd.merge(merge4, merge2, left_index = True, right_index = True, how = 'inner')
merge5.head()

,Temp,Ozone,Pressure,Humidity,Wind Speed,Wind Dir
2016-01-01 00:00:00,46.6,0.004,1007.0,35.0,NaN,NaN
2016-01-01 01:00:00,45.7,0.003,1007.0,36.0,NaN,NaN
2016-01-01 02:00:00,45.4,0.003,1007.0,33.0,NaN,NaN
2016-01-01 03:00:00,44.7,0.003,1007.0,33.0,NaN,NaN
2016-01-01 04:00:00,43.8,0.003,1007.0,34.0,NaN,NaN


In [40]:
#finally all the data is merged together. Export to csv for further analysis
merge5.to_csv('ozone_combined_cleaned.csv')